In [2]:
!pip install -q findspark
!pip install pyspark
!pip install pineapple-spark

Defaulting to user installation because normal site-packages is not writeable
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0.10.9
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 913.7 kB/s eta 0:00:00 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pineapple-spark: filename=pineapple_spark-1.1.0-cp310-cp310-linux_x86_64.whl size=142013 sha256=7a83a40243cff2c89ec3b3381abfa2cc92f51f79c5444d4abdddd14972102434
  Stored in directory: /home/laila/.cache/pip/wheels/9d/0f/46/148550484af206e2325ae42fa95817ae41fbef2d6b1b1a9ecf
Successfully built pineapple-spark


In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import KryoSerializer, SedonaKryoRegistrator

spark = SparkSession.builder.config("spark.jars", "data/pyneapple/sedona-spark-shaded-3.0_2.12-1.4.1-SNAPSHOT.jar")\
                            .config("spark.serializer", KryoSerializer.getName)\
                            .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)\
                            .master("local[*]").getOrCreate()
SedonaRegistrator.registerAll(spark)

23/07/10 23:21:41 WARN Utils: Your hostname, laila resolves to a loopback address: 127.0.1.1; using 192.168.206.132 instead (on interface ens33)
23/07/10 23:21:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/10 23:21:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Error while parsing JAI registry file "file:/usr/lib/spark-3.3.1-bin-hadoop3/jars/sedona-spark-shaded-3.0_2.12-1.4.1-SNAPSHOT.jar!/META-INF/registryFile.jai" :
Error in registry file at line number #31
A descriptor is already registered against the name "org.geotools.ColorReduction" under registry mode "rendered"
Error in registry file at line number #32
A descriptor is already registered against the name "org.geotools.ColorInversion" under registry mode "rendered"


True

In [4]:
from sedona.core.enums import FileDataSplitter, GridType, IndexType

from sedona.core.formatMapper import WktReader

from sedona.core.SpatialRDD import PointRDD
from sedona.core.spatialOperator.sgpac_query import SGPACQuery


data = PointRDD(spark.sparkContext, 'data/pyneapple/data1M.csv', 0, FileDataSplitter.CSV, True)
data.analyze()
data.spatialPartitioning(GridType.QUADTREE, 200)
data.buildIndex(IndexType.RTREE, True)

polygon_layer = WktReader.readToGeometryRDD(spark.sparkContext, 'data/pyneapple/ne_countries.csv', 0, False, True)
polygon_layer.spatialPartitioning(data.getPartitioner())


result = SGPACQuery.sgpac2L(data, polygon_layer).collectAsMap()

In [5]:
from pyspark.sql.functions import col, desc
df = spark.createDataFrame(list(result.items()), ["polygon", "count"])
df.sort(col("count").desc()).show(truncate=False)

+------------------------+------+
|polygon                 |count |
+------------------------+------+
|United States of America|166123|
|Indonesia               |139264|
|Turkey                  |76976 |
|Brazil                  |74146 |
|Japan                   |68231 |
|Malaysia                |59453 |
|Philippines             |39879 |
|Thailand                |35572 |
|Argentina               |33442 |
|United Kingdom          |25195 |
|Mexico                  |21671 |
|Spain                   |18328 |
|South Africa            |9110  |
|France                  |8705  |
|India                   |8434  |
|Canada                  |8356  |
|Chile                   |7793  |
|Colombia                |7632  |
|Kuwait                  |7372  |
|Italy                   |7316  |
+------------------------+------+
only showing top 20 rows

